In [18]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install jupyter
!pip install ta
!pip install lxml




  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.5-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached jupyterlab-4.3.4-py3-none-any.whl.metadata (16 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl.metadata (4.5 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.14.2-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.8 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 19.0 MB/s eta 0:00:00


In [2]:
!pip3 install html5lib


  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)


In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd
import time

# 設定抓取日期範圍
start_date = date(2020, 1, 1)
end_date = date(2024, 7, 30)

# 儲存每日抓取到的資料 DataFrame
dfs = []

current_date = start_date
while current_date <= end_date:
    # 將日期轉換成 TWSE 網址需要的格式 YYYYMMDD
    date_str = current_date.strftime("%Y%m%d")
    print(f"正在處理日期：{date_str}")

    url = f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={date_str}&type=ALLBUT0999&response=html'

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # 確保 HTTP 狀態碼為 200
    except Exception as e:
        print(f"日期 {date_str} 抓取資料時發生錯誤：{e}")
        current_date += timedelta(days=1)
        continue

    # 用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找出所有表格
    tables = soup.find_all('table')
    target_table = None

    # 根據表格中是否包含關鍵字來判斷是否是目標表格
    for table in tables:
        if "每日收盤行情" in table.text:
            target_table = table
            break

    # 如果找到表格則進一步擷取資料
    if target_table:
        rows = target_table.find_all('tr')
        # 若表格第一列為表頭 (有 <th>)，則取出作為欄位名稱
        header_cells = rows[0].find_all('th') if rows else []
        headers = [cell.text.strip() for cell in header_cells] if header_cells else None

        # 從資料列開始讀取資料
        data_list = []
        # 如果有表頭，則從第二列開始，否則從第一列開始
        start_idx = 1 if headers else 0
        for row in rows[start_idx:]:
            cells = row.find_all('td')
            # 若該行沒有資料則跳過
            if not cells:
                continue
            row_data = [cell.text.strip() for cell in cells]
            data_list.append(row_data)

        # 若當天有資料，再建立 DataFrame 並記錄日期
        if data_list:
            if headers and len(headers) == len(data_list[0]):
                df_day = pd.DataFrame(data_list, columns=headers)
            else:
                # 若無法取得標題或資料行數與標題數不符，則使用預設欄位名稱
                df_day = pd.DataFrame(data_list)
            # 將日期加入一欄
            df_day['date'] = current_date
            dfs.append(df_day)
            print(f"日期 {date_str} 資料抓取成功，資料筆數：{len(df_day)}")
        else:
            print(f"日期 {date_str} 找到目標表格，但沒有資料列。")
    else:
        print(f"日期 {date_str} 未找到目標表格！")

    # 為避免頻繁請求，延遲 1 秒
    time.sleep(1)
    current_date += timedelta(days=1)

# 若有資料，整合成一個 DataFrame
if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
    # 將日期欄轉為 datetime 格式
    df_total['date'] = pd.to_datetime(df_total['date'])
    # 可依日期排序
    df_total.sort_values(by='date', inplace=True)
    print("全部資料整理完成！")
    print(df_total.head())
# 匯出為 CSV
    file_name = "每日收盤行情_20200101_20240730.csv"
    df_total.to_csv(file_name, index=False, encoding='utf-8-sig')  # 使用 utf-8-sig 編碼避免中文亂碼
    print(f"資料已成功匯出為 CSV 檔案：{file_name}")
else:
    print("沒有抓取到任何資料！")



正在處理日期：20200101
日期 20200101 未找到目標表格！
正在處理日期：20200102
日期 20200102 資料抓取成功，資料筆數：1114
正在處理日期：20200103
日期 20200103 資料抓取成功，資料筆數：1114
正在處理日期：20200104
日期 20200104 未找到目標表格！
正在處理日期：20200105
日期 20200105 未找到目標表格！
正在處理日期：20200106
日期 20200106 資料抓取成功，資料筆數：1114
正在處理日期：20200107
日期 20200107 資料抓取成功，資料筆數：1114
正在處理日期：20200108
日期 20200108 資料抓取成功，資料筆數：1114
正在處理日期：20200109
日期 20200109 資料抓取成功，資料筆數：1111
正在處理日期：20200110
日期 20200110 資料抓取成功，資料筆數：1111
正在處理日期：20200111
日期 20200111 未找到目標表格！
正在處理日期：20200112
日期 20200112 未找到目標表格！
正在處理日期：20200113
日期 20200113 資料抓取成功，資料筆數：1111
正在處理日期：20200114
日期 20200114 資料抓取成功，資料筆數：1111
正在處理日期：20200115
日期 20200115 資料抓取成功，資料筆數：1111
正在處理日期：20200116
日期 20200116 資料抓取成功，資料筆數：1111
正在處理日期：20200117
日期 20200117 資料抓取成功，資料筆數：1111
正在處理日期：20200118
日期 20200118 未找到目標表格！
正在處理日期：20200119
日期 20200119 未找到目標表格！
正在處理日期：20200120
日期 20200120 資料抓取成功，資料筆數：1114
正在處理日期：20200121
日期 20200121 未找到目標表格！
正在處理日期：20200122
日期 20200122 未找到目標表格！
正在處理日期：20200123
日期 20200123 未找到目標表格！
正在處理日期：20200124
日期 20200124 未找到目標表格！
正在處理日期：2

In [20]:
import pandas as pd
import numpy as np

def clean_and_process_data(df):
    # 1. **重新命名欄位**
    df.columns = [
        "證券代號", "證券名稱", "成交股數", "成交筆數", "成交金額", 
        "開盤價", "最高價", "最低價", "收盤價", "漲跌符號", 
        "漲跌價差", "最後揭示買價", "最後揭示買量", 
        "最後揭示賣價", "最後揭示賣量", "本益比", "date"
    ]
    
    # 2. **排除 ETF、權證等非普通股票標的**
    # 假設 ETF、權證等代號開頭是 "00"、"006" 等
    df = df[~df["證券代號"].str.startswith(("00", "006","01","02"))]

    # 3. **去除千位逗號並轉為數值型別**
    numeric_cols = ["成交股數", "成交金額", "開盤價", "最高價", "最低價", "收盤價", "漲跌價差", "最後揭示買價", "最後揭示賣價", "本益比"]
    for col in numeric_cols:
        df[col] = (
            df[col].astype(str)
                  .str.replace(',', '', regex=True)  # 去除逗號
                  .replace(['---', '--'], np.nan)  # 處理無交易或停牌符號為 NaN
                  .astype(float)  # 轉為 float 型別
        )

    # 4. **日期格式轉換**
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

    # 5. **處理漲跌符號**
    df["漲跌符號"] = df["漲跌符號"].replace({'X': 0, '+': 1, '-': -1}).fillna(0)  # 無交易則記為 0
    df["漲跌價差"] = df["漲跌價差"].fillna(0)  # 若為 NaN，當作無變動

    # 6. **濾除極端或異常值**
    df = df.dropna(subset=["收盤價"])  # 刪除收盤價為 NaN 的資料
    df = df[(df["收盤價"] > 0) & (df["最高價"] > 0) & (df["最低價"] > 0)]  # 價格需為正數

    # 7. **標註停牌或無交易資料的證券**
    suspended_or_no_data = df[df["成交金額"].isna()]["證券代號"].unique()
    print(f"停牌或無交易資料的證券代號：{suspended_or_no_data}")

    # 8. **匯出清理後的 CSV**
    df.to_csv("清理後每日收盤行情.csv", index=False, encoding='utf-8-sig')
    print("資料清理完成，已匯出為 CSV 檔案！")

    return df

# 測試清理函式
input_path = "每日收盤行情_20200101_20240730.csv"
df = pd.read_csv(input_path)
cleaned_df = clean_and_process_data(df)
print(cleaned_df.head())


C:\Users\Administrator\AppData\Local\Temp\ipykernel_44796\3602485641.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (
C:\Users\Administrator\AppData\Local\Temp\ipykernel_44796\3602485641.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (
C:\Users\Administrator\AppData\Local\Temp\ipykernel_44796\3602485641.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

停牌或無交易資料的證券代號：[]
資料清理完成，已匯出為 CSV 檔案！
   證券代號 證券名稱       成交股數   成交筆數          成交金額     開盤價     最高價     最低價    收盤價  \
1  3338   泰碩  4717909.0  3,095  4.533602e+08   95.60   96.70   95.10   96.0   
2  3346   麗清    82460.0     54  2.505811e+06   30.35   30.65   30.35   30.5   
3  3356   奇偶   184297.0    137  5.521845e+06   30.00   30.10   29.80   29.9   
4  3376  新日興  7729554.0  4,711  1.050449e+09  129.00  140.00  129.00  139.0   
5  3380   明泰   875096.0    474  2.064470e+07   23.65   23.80   23.50   23.5   

   漲跌符號   漲跌價差  最後揭示買價 最後揭示買量  最後揭示賣價 最後揭示賣量    本益比       date  
1   1.0   1.30    96.0      5   96.10     30  30.48 2020-01-02  
2   1.0   0.15    30.4      1   30.50      6  33.89 2020-01-02  
3  -1.0   0.10    29.9      1   30.00     15  43.97 2020-01-02  
4   1.0  10.00   138.5     28  139.00    134  19.09 2020-01-02  
5  -1.0   0.05    23.5     88   23.55      6  44.34 2020-01-02  


In [24]:
df = cleaned_df.sort_values(by="證券代號", ascending=True)
df = df[~df["證券代號"].str.startswith(("02","01"))]
df.head(20)
df.describe

<bound method NDFrame.describe of          證券代號 證券名稱        成交股數   成交筆數          成交金額     開盤價     最高價     最低價  \
1164855  1101   台泥  17159173.0  9,874  5.481136e+08   32.05   32.15   31.85   
115365   1101   台泥  22750386.0  9,742  9.990143e+08   43.75   44.15   43.50   
1004775  1101   台泥   9774757.0  8,264  3.606628e+08   37.20   37.25   36.80   
1052826  1101   台泥  15884461.0  6,869  5.207745e+08   32.75   33.10   32.70   
432300   1101   台泥  23552465.0  9,940  1.271178e+09   54.20   54.50   53.60   
...       ...  ...         ...    ...           ...     ...     ...     ...   
10378    9958  世紀鋼   1702631.0  1,063  1.222405e+08   72.50   72.50   71.30   
615590   9958  世紀鋼   7398825.0  4,760  8.512327e+08  111.00  117.00  111.00   
966129   9958  世紀鋼   1340962.0    898  1.502732e+08  110.50  113.50  110.50   
1022507  9958  世紀鋼   5241359.0  3,155  8.376997e+08  158.50  161.50  158.50   
1292892  9958  世紀鋼   7602686.0  5,936  1.664360e+09  221.00  222.50  212.50   

            收盤價  

In [23]:
import pandas as pd
import numpy as np
import ta  # 技術指標套件

# 技術指標計算函式
def add_technical_factors(df):
    df = df.sort_values(["證券代號", "date"])  # 先依證券代號與日期排序

    # 1. **移動平均線 (MA)**
    df["MA5"] = df.groupby("證券代號")["收盤價"].transform(lambda x: x.rolling(window=5).mean())  # 5日均線
    df["MA10"] = df.groupby("證券代號")["收盤價"].transform(lambda x: x.rolling(window=10).mean())  # 10日均線
    df["MA20"] = df.groupby("證券代號")["收盤價"].transform(lambda x: x.rolling(window=20).mean())  # 20日均線
    df["MA60"] = df.groupby("證券代號")["收盤價"].transform(lambda x: x.rolling(window=60).mean())  # 60日均線

    # 2. **RSI (相對強弱指標)**，窗口為 14 天
    df["RSI14"] = df.groupby("證券代號").apply(
        lambda x: ta.momentum.rsi(x["收盤價"], window=14)
    ).reset_index(level=0, drop=True)

    # 3. **成交量動能因子**
    df["vol_ma5"] = df.groupby("證券代號")["成交股數"].transform(lambda x: x.rolling(window=5).mean())  # 5日平均成交量
    df["vol_ma20"] = df.groupby("證券代號")["成交股數"].transform(lambda x: x.rolling(window=20).mean())  # 20日平均成交量
    df["vol_ratio"] = df["vol_ma5"] / df["vol_ma20"]  # 成交量比率 (5日均量 / 20日均量)

    # 4. **動能因子 (Momentum)**，收盤價變動
    df["momentum_10d"] = df.groupby("證券代號")["收盤價"].transform(lambda x: x.pct_change(periods=10))  # 10日動能

    # 5. **報酬率因子 (Returns)**：
    # 隔日報酬率 (t+1)
    df["return_t+1"] = df.groupby("證券代號")["收盤價"].shift(-1) / df["收盤價"] - 1

    # N 日報酬率 (7 日、14 日)
    df["return_t+7"] = df.groupby("證券代號")["收盤價"].shift(-7) / df["收盤價"] - 1
    df["return_t+14"] = df.groupby("證券代號")["收盤價"].shift(-14) / df["收盤價"] - 1

    # 6. **分類標籤 (漲 / 跌標籤)**：1 表示漲，0 表示跌
    df["y_label_t+1"] = (df["return_t+1"] > 0).astype(int)  # 隔日漲為 1，跌為 0
    df["y_label_t+7"] = (df["return_t+7"] > 0).astype(int)  # 7 日後漲為 1，跌為 0

    return df

# 測試因子建構
df = pd.read_csv("清理後每日收盤行情.csv")  # 載入清理後的資料
df = add_technical_factors(df)

# 查看新增的技術指標欄位
print(df.head())


      證券代號 證券名稱        成交股數   成交筆數         成交金額    開盤價    最高價    最低價    收盤價  \
590   1101   台泥  18470566.0  6,251  813465904.0  43.80  44.15  43.80  44.10   
1557  1101   台泥  18387114.0  6,733  807459827.0  44.15  44.25  43.45  43.95   
2525  1101   台泥  13867625.0  5,566  602752323.0  43.50  43.70  43.30  43.45   
3494  1101   台泥  14185347.0  4,739  616059913.0  43.45  43.60  43.20  43.60   
4462  1101   台泥  13465670.0  4,685  583893343.0  43.15  43.55  43.15  43.40   

      漲跌符號  ...  RSI14     vol_ma5 vol_ma20  vol_ratio momentum_10d  \
590    1.0  ...    NaN         NaN      NaN        NaN          NaN   
1557  -1.0  ...    NaN         NaN      NaN        NaN          NaN   
2525  -1.0  ...    NaN         NaN      NaN        NaN          NaN   
3494   1.0  ...    NaN         NaN      NaN        NaN          NaN   
4462  -1.0  ...    NaN  15675264.4      NaN        NaN          NaN   

      return_t+1 return_t+7  return_t+14  y_label_t+1  y_label_t+7  
590    -0.003401   0.003401  